In [1]:
import pandas as pd
import numpy as np
import os
import datetime
import pickle
import seaborn as sns
import pickle

### EDA :

In [2]:
path = './data/'
print('list of files within folder  :',os.listdir(path))
df_meta = pd.read_csv(path+'articles_metadata.csv')
print('dataframe info :',df_meta.info())
print('##----------------------------------##')
print('shape of "articles_metadata.csv" file  :',df_meta.shape)
print('duplicated rows on "article_id" subset :',df_meta.duplicated(subset='article_id', keep='first').sum())
print('Articles count from min to max:',df_meta['article_id'].min(), df_meta['article_id'].max())
day_divider = 86400000
df_meta['creation_time'] = df_meta['created_at_ts'].values.astype(dtype='datetime64[ms]')
print('Max and min dates of dataset   :',df_meta['creation_time'].max(),'       ', df_meta['creation_time'].min())
delta_time = (df_meta['created_at_ts'].max()-df_meta['created_at_ts'].min())/day_divider/365
print('Time  interval of dataset  : %.2f'%delta_time)
print('##----------------------------------##')
print('File : Articles metadata.csv')
df_meta.head(3)

list of files within folder  : ['articles_embeddings.pickle', 'articles_metadata.csv', 'clicks', 'clicks.zip', 'clicks_sample.csv']
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 364047 entries, 0 to 364046
Data columns (total 5 columns):
 #   Column         Non-Null Count   Dtype
---  ------         --------------   -----
 0   article_id     364047 non-null  int64
 1   category_id    364047 non-null  int64
 2   created_at_ts  364047 non-null  int64
 3   publisher_id   364047 non-null  int64
 4   words_count    364047 non-null  int64
dtypes: int64(5)
memory usage: 13.9 MB
dataframe info : None
##----------------------------------##
shape of "articles_metadata.csv" file  : (364047, 5)
duplicated rows on "article_id" subset : 0
Articles count from min to max: 0 364046
Max and min dates of dataset   : 2018-03-13 12:12:30         2006-09-27 11:14:35
Time  interval of dataset  : 11.47
##----------------------------------##
File : Articles metadata.csv


,article_id,category_id,created_at_ts,publisher_id,words_count,creation_time
0,0,0,1513144419000,0,168,2017-12-13 05:53:39
1,1,1,1405341936000,0,189,2014-07-14 12:45:36
2,2,1,1408667706000,0,250,2014-08-22 00:35:06


In [59]:
print('check distribution of articles creation date :')
df_meta['creation_time'].value_counts(bins=6)

check distribution of words count :


(2016-04-15 00:02:50.833333248, 2018-03-13 12:12:30]              243418
(2014-05-18 11:53:11.666666752, 2016-04-15 00:02:50.833333248]     88834
(2012-06-19 23:43:32.500000, 2014-05-18 11:53:11.666666752]        31340
(2010-07-23 11:33:53.333333248, 2012-06-19 23:43:32.500000]          352
(2008-08-24 23:24:14.166666752, 2010-07-23 11:33:53.333333248]        54
(2006-09-23 06:48:07.524999935, 2008-08-24 23:24:14.166666752]        49
Name: creation_time, dtype: int64

In [58]:
df_meta['publisher_id'].value_counts()

0    364047
Name: publisher_id, dtype: int64

In [5]:
print('check distribution of words count :')
df_meta['words_count'].value_counts(bins=4)

check distribution of words count :


(-6.691000000000001, 1672.5]    364022
(1672.5, 3345.0]                    22
(3345.0, 5017.5]                     2
(5017.5, 6690.0]                     1
Name: words_count, dtype: int64

In [5]:
af = pd.read_csv(path+'clicks_sample.csv')
print('dataframe infos :',af.info())
print('##----------------------------------##')
print('Dataframe columns :',af.columns)
print('##----------------------------------##')
print(af.shape)
af.head(3)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1883 entries, 0 to 1882
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype
---  ------               --------------  -----
 0   user_id              1883 non-null   int64
 1   session_id           1883 non-null   int64
 2   session_start        1883 non-null   int64
 3   session_size         1883 non-null   int64
 4   click_article_id     1883 non-null   int64
 5   click_timestamp      1883 non-null   int64
 6   click_environment    1883 non-null   int64
 7   click_deviceGroup    1883 non-null   int64
 8   click_os             1883 non-null   int64
 9   click_country        1883 non-null   int64
 10  click_region         1883 non-null   int64
 11  click_referrer_type  1883 non-null   int64
dtypes: int64(12)
memory usage: 176.7 KB
dataframe infos : None
##----------------------------------##
Dataframe columns : Index(['user_id', 'session_id', 'session_start', 'session_size',
       'click_article_id', 'click

,user_id,session_id,session_start,session_size,click_article_id,click_timestamp,click_environment,click_deviceGroup,click_os,click_country,click_region,click_referrer_type
0,0,1506825423271737,1506825423000,2,157541,1506826828020,4,3,20,1,20,2
1,0,1506825423271737,1506825423000,2,68866,1506826858020,4,3,20,1,20,2
2,1,1506825426267738,1506825426000,2,235840,1506827017951,4,1,17,1,16,2


### Concatenate all click-files to define one globale dataframe 

In [50]:
# Define all click files main path
path_click = path+'clicks/'
list_clicks = os.listdir(path_click)

trames=[]
i = 0
for file in list_clicks:
    df = pd.read_csv(path_click+file)
    df['session_file_source'] = file
    trames.append(df)
    i = i+1
    
# Concatenate all files
global_df = pd.concat(trames)

# get out some specs from global file
print('shape of global file :',global_df.shape)
print('columns :',list(global_df.columns))


shape of global file : (2988181, 13)
columns : ['user_id', 'session_id', 'session_start', 'session_size', 'click_article_id', 'click_timestamp', 'click_environment', 'click_deviceGroup', 'click_os', 'click_country', 'click_region', 'click_referrer_type', 'session_file_source']


In [51]:
# Save file to csv format
global_df.to_csv('all_clicks_files.csv',index=False)

In [6]:
global_df = pd.read_csv('all_clicks_files.csv')
global_df.head(2)

,user_id,session_id,session_start,session_size,click_article_id,click_timestamp,click_environment,click_deviceGroup,click_os,click_country,click_region,click_referrer_type,session_file_source
0,0,1506825423271737,1506825423000,2,157541,1506826828020,4,3,20,1,20,2,clicks_hour_000.csv
1,0,1506825423271737,1506825423000,2,68866,1506826858020,4,3,20,1,20,2,clicks_hour_000.csv


In [7]:
print('Unique device group  :',global_df['click_deviceGroup'].unique())
print('Unique environment  :',global_df['click_environment'].unique())
print('Unique session-size  :',global_df['session_size'].unique())
print('Unique OS  :',global_df['click_os'].unique())
print('Unique region  :',global_df['click_region'].unique())
print('Unique country  :',global_df['click_country'].unique())
print('Unique referrer type  :',global_df['click_referrer_type'].unique())

Unique device group  : [3 1 4 5 2]
Unique environment  : [4 2 1]
Unique session-size  : [  2   3   7   6   4   5  16   8  10   9  24  11  13  20  12  14  15  42
  18  51  26  22  19  27  23  25  21  31  36  37  30  32  17  35  39  29
  28  82  57  41  33  75  53  34  52  38  46 106  74  47  71  58  65  98
  72  59  43  60  40  48  86  62 124  92  68  44  45  67 107  94  79  56]
Unique OS  : [20 17  2 12 13 19  5  3]
Unique region  : [20 16 24 21 17 25 12 18 19  6  9 13  8  7 28  5  4 26 15 11 10 27  2  1
 14 22  3 23]
Unique country  : [ 1 11 10  8  2  4  7  3  9  5  6]
Unique referrer type  : [2 1 5 7 6 4 3]


In [8]:
# Ratio of seen atricles within global database
print('Count of consulted articles :',len(global_df['click_article_id'].unique()),'which represents a ratio of : %.2f'%(100*len(global_df['click_article_id'].unique())/df_meta.shape[0]),'%')

Count of consulted articles : 46033 which represents a ratio of : 12.64 %


#### Build popularity recommander 


It is a type of recommendation system which works on the principle of popularity and or anything which is in trend. These systems check about the product or movie which are in trend or are most popular among the users and directly recommend those.

For example, if a product is often purchased by most people then the system will get to know that that product is most popular so for every new user who just signed it, the system will recommend that product to that user also and chances becomes high that the new user will also purchase that.


In [9]:
# Create popularity model
def get_popularity_rec(clicks, n_reco=5):
    # Compute the five most popular articles
    df_popularity = clicks.groupby(by=['click_article_id'])['click_timestamp'].count().sort_values(ascending=False).reset_index()
    df_popularity.rename(columns = {'user_id':'popularity'}, inplace=True)
    return df_popularity.click_article_id.head(n_reco).to_list()

In [10]:
# Call the function to get the most viewed articles according to users activity
# Can be used for new users with no history records on activity
get_popularity_rec(global_df, n_reco=5)

[160974, 272143, 336221, 234698, 123909]

#### Get the ratings for each article :
create a function that will calculate the implicit ratings

In [5]:
df = pd.read_csv('all_clicks_files.csv')
df.drop(['session_size','click_environment','click_deviceGroup','click_os','click_country','click_region','click_referrer_type'],axis=1,inplace=True)
df.head(2)

,user_id,session_id,session_start,click_article_id,click_timestamp,session_file_source
0,0,1506825423271737,1506825423000,157541,1506826828020,clicks_hour_000.csv
1,0,1506825423271737,1506825423000,68866,1506826858020,clicks_hour_000.csv


In [25]:
df.drop('session_start',axis=1,inplace=True)

In [26]:
df.head(2)

,user_id,session_id,click_article_id,click_timestamp
0,0,1506825423271737,157541,1506826828020
1,0,1506825423271737,68866,1506826858020


In [27]:
df.to_csv('clicks.csv',index=False)

In [6]:
def get_ratings(clicks):
    ### Compute the rating dataframe providing for each interaction a rating based on the number of clicks per article weighted by the total number of clicks per user
    
    # Create a dataframe containing the number of clicks for each user and each article    
    count_clicks_by_articles_by_user = clicks.groupby(["user_id", "click_article_id"]).agg(count_clicks_by_articles_by_user=("session_id", "count"))
    
    # Create a dataframe containing the number of clicks for each user   
    count_clicks_by_user = clicks.groupby(["user_id"]).agg(count_clicks_by_user=("session_id", "count"))
    
    # Compute the weighted ratio of clicks
    clicks_count = count_clicks_by_articles_by_user.join(count_clicks_by_user, on="user_id")
    clicks_count['rating'] = clicks_count["count_clicks_by_articles_by_user"] / clicks_count["count_clicks_by_user"]
    
    # Just rename columns
    ratings = clicks_count.reset_index().drop(["count_clicks_by_articles_by_user","count_clicks_by_user"], axis=1).rename(columns={"click_article_id":"article_id"})
    
    return ratings

In [13]:
ratings = get_ratings(df)
ratings

,user_id,article_id,rating
0,0,68866,0.125
1,0,87205,0.125
2,0,87224,0.125
3,0,96755,0.125
4,0,157541,0.125
...,...,...,...
2950705,322894,168401,0.500
2950706,322895,63746,0.500
2950707,322895,289197,0.500
2950708,322896,30760,0.500


In [75]:
# Create a train and test splits for modeling
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(ratings, train_size=0.8, random_state=31)

In [14]:
# Install Implicit library
!pip install implicit==0.6.1
!pip install ipywidgets

In [4]:
from scipy.sparse import csr_matrix
from sklearn.model_selection import train_test_split

from tqdm import tqdm

from implicit.als import AlternatingLeastSquares
from implicit.bpr import BayesianPersonalizedRanking
from implicit.lmf import LogisticMatrixFactorization
from implicit.evaluation import precision_at_k, mean_average_precision_at_k, ndcg_at_k, AUC_at_k

from time import time

To understand the metrics used :

https://machinelearninginterview.com/topics/machine-learning/mapatk_evaluation_metric_for_ranking/

In [20]:
def train_models(train_df, test_df, models_list, n_recommandations):
    
    '''
    Train recommendation models and provide evaluation metrics.

            Parameters:
                    train_df (Pandas Dataframe): Dataframe containing training interactions datas
                    test_df (Pandas Dataframe): Dataframe containing test interactions datas
                    models_list (List) : Python List containing models to be evaluated
                    n_recommandations (Int) : Integer representing the number of recommandations to provide


            Returns:
                    df_results (Pandas Dataframe): Dataframe containing the results of evaluation performed on the models
    '''
    
    # First, let's initialize a blank results dataframe 
    # Each column correspond to the evaluation metrics to use
    df_results = pd.DataFrame(columns=['model', 'Precision@k','MAP@k','nDCG@k',"train_time"])
    
    # Get dimensions of the sparse matrix    
    dim = (max(train_df.user_id.max(),test_df.user_id.max())+1, max(train_df.article_id.max(),test_df.article_id.max())+1)
    
    # Creating sparse matrix for train and test dataframe
    # Why creating sparse matrix ? : because it is the data format expected by Implicit
    train_csr = csr_matrix((train_df['rating'], (train_df['user_id'], train_df['article_id'])), dim)
    test_csr = csr_matrix((test_df['rating'], (test_df['user_id'], test_df['article_id'])), dim)
    
    # Iterate over each models to evaluate in the list
    for model in models_list:
        
        print("##"*30)
        print("[INFO] : Start training the model : ", model.__class__.__name__)
        
        # Launch the timer
        train_start_time = time()
        
        # Train the choosen model
        model.fit(train_csr)
        
        # Stop the timer and calculate the training time
        train_time = time() - train_start_time
              
        # Calculate evaluation metrics        
        precision_k = round(precision_at_k(model, train_csr, test_csr), 5)
        map_at_k = round(mean_average_precision_at_k(model, train_csr, test_csr), 5)
        ndcg_k = round(ndcg_at_k(model, train_csr, test_csr), 5)
        print("[INFO] : Precision@k = ", precision_k)
        print("[INFO] : MAP@k = ", map_at_k)
        print("[INFO] : nDCG@k = ", ndcg_k)
        print("##"*30)
        
        # Log results in the results dataframe
        
        df_results = df_results.append({
            'model': model.__class__.__name__,
            'Precision@k': precision_k,
            'MAP@k': map_at_k,
            'nDCG@k': ndcg_k,
            'train_time': round(train_time,5),
        }, ignore_index=True)
        
    return df_results

In [21]:
# Define models
models_list = [AlternatingLeastSquares(), BayesianPersonalizedRanking(), LogisticMatrixFactorization()]

In [84]:
# Launch models training 
train_models(train_df          = train_df,
             test_df           = test_df, 
             models_list       = models_list, 
             n_recommandations = 5)

############################################################
[INFO] : Start training the model :  AlternatingLeastSquares


100%|██████████| 209671/209671 [06:24<00:00, 545.28it/s]
C:\Users\AMC\AppData\Local\Temp\ipykernel_22604\4285575009.py:55: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_results = df_results.append({


[INFO] : Precision@k =  0.10099
[INFO] : MAP@k =  0.06523
[INFO] : nDCG@k =  0.0932
############################################################
############################################################
[INFO] : Start training the model :  BayesianPersonalizedRanking


100%|██████████| 209671/209671 [05:38<00:00, 619.52it/s]
C:\Users\AMC\AppData\Local\Temp\ipykernel_22604\4285575009.py:55: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_results = df_results.append({


[INFO] : Precision@k =  0.12337
[INFO] : MAP@k =  0.08388
[INFO] : nDCG@k =  0.11816
############################################################
############################################################
[INFO] : Start training the model :  LogisticMatrixFactorization


100%|██████████| 209671/209671 [05:51<00:00, 597.31it/s]

[INFO] : Precision@k =  0.0313
[INFO] : MAP@k =  0.01085
[INFO] : nDCG@k =  0.01985
############################################################



C:\Users\AMC\AppData\Local\Temp\ipykernel_22604\4285575009.py:55: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_results = df_results.append({


,model,Precision@k,MAP@k,nDCG@k,train_time
0,AlternatingLeastSquares,0.10099,0.06523,0.09320,309.36034
1,BayesianPersonalizedRanking,0.12337,0.08388,0.11816,75.19970
2,LogisticMatrixFactorization,0.03130,0.01085,0.01985,52.10852


#### Train with a ratio of global df

In [22]:
SAMPLE_RATIO = 0.05
ratings_sample = ratings.sample(frac=SAMPLE_RATIO, random_state=8989)
print(ratings_sample.shape)
ratings_sample.head()

(147536, 3)


,user_id,article_id,rating
2936274,317101,199393,0.500000
1620248,99395,59758,0.500000
612201,31691,293050,0.016393
1525674,90863,32750,0.076923
1100500,60312,236524,0.050000


In [23]:
train_df_ratio, test_df_ratio = train_test_split(ratings_sample, train_size=0.8, random_state=31)

In [24]:
print(train_df_ratio.shape)
print(test_df_ratio.shape)

(118028, 3)
(29508, 3)


In [29]:
train_models(train_df          = train_df_ratio,
             test_df           = test_df_ratio, 
             models_list       = models_list, 
             n_recommandations = 5)

############################################################
[INFO] : Start training the model :  AlternatingLeastSquares


  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/25966 [00:00<?, ?it/s]

  0%|          | 0/25966 [00:00<?, ?it/s]

  0%|          | 0/25966 [00:00<?, ?it/s]

...

  0%|          | 0/25966 [00:00<?, ?it/s]

  0%|          | 0/25966 [00:00<?, ?it/s]

[INFO] : Precision@k =  0.06721
[INFO] : MAP@k =  0.02346
[INFO] : nDCG@k =  0.03473
############################################################


C:\Users\AMC\AppData\Local\Temp\ipykernel_9920\4285575009.py:55: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_results = df_results.append({


,model,Precision@k,MAP@k,nDCG@k,train_time
0,AlternatingLeastSquares,0.01732,0.00594,0.00874,64.59176
1,BayesianPersonalizedRanking,0.00288,0.00125,0.00165,5.96202
2,LogisticMatrixFactorization,0.06721,0.02346,0.03473,12.17533


The most efficient model is LaogisticMatrixFactorisation and we will use it for further modeling

In [17]:
df.head(2)

,user_id,session_id,session_start,click_article_id,click_timestamp,session_file_source
0,0,1506825423271737,1506825423000,157541,1506826828020,clicks_hour_000.csv
1,0,1506825423271737,1506825423000,68866,1506826858020,clicks_hour_000.csv


In [2]:
import pickle

def compute_interaction_matrix(clicks):
    # Create interaction DF (count of interactions between users and articles)
    interactions = clicks.groupby(['user_id','click_article_id']).size().reset_index(name='count')
    # print('Interactions DF shape: ', interactions.shape)

    # csr = compressed sparse row (good format for math operations with row slicing)
    # Create sparse matrix of shape (number_items, number_user)
    csr_item_user = csr_matrix((interactions['count'].astype(float),
                                (interactions['click_article_id'],
                                 interactions['user_id'])))
    # print('CSR Shape (number_items, number_user): ', csr_item_user.shape)
    
    # Create sparse matrix of shape (number_user, number_items)
    csr_user_item = csr_matrix((interactions['count'].astype(float),
                                (interactions['user_id'],
                                 interactions['click_article_id'])))
    # print('CSR Shape (number_user, number_items): ', csr_user_item.shape)
    
    return csr_item_user, csr_user_item

In [17]:
def get_cf_reco(clicks, userID, csr_item_user, csr_user_item, model_path=None, n_reco=5, train=True):
    
    start = time()
    # Train the model on sparse matrix of shape (number_items, number_user)
    
    if train or model_path is None:
        model = LogisticMatrixFactorization(factors= 128, random_state=42)
        print("[INFO] : Start training model")
        model.fit(csr_user_item)

        # Save model to disk
        with open('recommender.model', 'wb') as filehandle:
            pickle.dump(model, filehandle)
    else:
        with open('recommender.model', 'rb') as filehandle:
            model = pickle.load(filehandle)

    # Recommend N best items from sparse matrix of shape (number_user, number_items)
    # Implicit built-in method
    # N (int) : number of results to return
    # filter_already_liked_items (bool) : if true, don't return items present in 
    # the training set that were rated/viewd by the specified user
    recommendations_list = []
    recommendations = model.recommend(userID, csr_user_item[userID], N=n_reco, filter_already_liked_items=True)

    print(f'[INFO] : Completed in {round(time() - start, 2)}s')
    
    #recommendations = [elt[0] for elt in recommendations]
    
    return recommendations

In [16]:
# Call the function, train the model and perform recommandations
userID = 61691
csr_item_user, csr_user_item = compute_interaction_matrix(df)

In [18]:
# Construct the prediction model
get_cf_reco(df, userID, csr_item_user, csr_user_item,model_path=None, n_reco=5, train=True)

[INFO] : Start training model


  0%|          | 0/30 [00:00<?, ?it/s]

[INFO] : Completed in 584.71s


(array([ 84942, 292390, 157332, 209348, 208508]),
 array([11.138317 ,  8.384117 ,  7.927927 ,  7.6736536,  7.5536165],
       dtype=float32))

In [21]:
# Call the function, load the model and perform recommandations
userID = 34
get_cf_reco(df, userID, csr_item_user, csr_user_item, model_path="./recommender.model", n_reco=5, train=False)

[INFO] : Completed in 0.35s


(array([354086, 158535, 348111,  30730, 199376]),
 array([10.955527 , 10.441811 ,  9.946945 ,  9.7908945,  9.310896 ],
       dtype=float32))

## Deal with Embedding File
https://towardsdatascience.com/visualizing-word-embedding-with-pca-and-t-sne-961a692509f5

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import PCA

In [52]:
# Load embedding file
with open('./data/articles_embeddings.pickle','rb') as pickle_file:
    emb_matrix = pickle.load(pickle_file)
print('Embedding shape  :',emb_matrix.shape)
print('Embedding type  :',type(emb_matrix))

Embedding shape  : (364047, 250)
Embedding type  : <class 'numpy.ndarray'>


In [51]:
# Choosen article has an index comprised in [0, 364047] 
print('Pick an article')
art = int(input('Give an interger from 0 to 364047'))
print('Article choosen is   :', art)

Pick an article
Article choosen is   : 88


In [144]:
# Test cosine similarity for 2 vectors within Embedding matrix
d=[]
l=[]
c = 0
art_vect = emb_matrix[art]
art_vect = np.reshape(art_vect,(1,-1))
print('choosen article :',art)

for i in range (100):
    if i == art :
        pass
    else :
        #print(' index = ',i)
        l = emb_matrix[i]
        l = np.reshape(l,(1,-1))
                
        cosine = cosine_similarity(l,art_vect)
         

        d.append(
            {
                'article_index':i,
                'cosine':cosine[0][0]
            
            }
        )

k=pd.DataFrame(d)
k.sort_values(by=['cosine'],ascending=False).head(5)

choosen article : 88


,article_index,cosine
79,79,0.862483
9,9,0.832213
61,61,0.825840
76,76,0.817524
89,90,0.807653
